
## Training and Pruning the ResNet 8 model to fit into the mobile size device.

**Project's Objective:**The project involves the traning and pruning of the model to integrate into a mobile application. The project aims to provide a good and compact image classification model to let people use it without relying on the server or internet connection. It will be really useful for Afrian people in remote places to use the app and utilize the power of machine learning for their tree-related activities (e.g gardening, harvesting, planting and using trees for herbal use).

I am very excited to work on this project for several reasons:
- 1) I can learn the architecture of the ResNet 8 for image classification. I will be involved pretty heavily into training, pruning and integrating the model into mobile application. The last stage is something I have never done before.
- 2) I can learn about the field of herbal medicine and the federated learning. This will be the first step into building the federated learning architecture where the weights of the model will be sent to the mobile application and the data will be trained within the mobile apps instead of sending that sensitive data back to the server.


### First Stage: The first stage involves loading the data, understand the ResNet 8 architecture and train the model with that architecture. (Everything will be in the Jupyter Notebook)

- Loading the data from google drive. ✅
- Understand ResNet 8 architecture
  [Resource 5min](https://towardsdatascience.com/understanding-and-visualizing-resnets-442284831be8)
- Using RestNet 8 for feature extraction ✅
- Doing dimensionality reduction with autoencoder
- Applying random forest


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/Thang_folder/Herb_Images_Dataset_shortlisted'


Mounted at /content/gdrive


In [ ]:
with open(f'{gdrive_path}/Acacia senegal/1-s2.0-S0268005X17306112-egi10HGQM8J72N.jpg') as file:
  print(file)

<_io.TextIOWrapper name='/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/Thang_folder/Herb_Images_Dataset_shortlisted/Acacia senegal/1-s2.0-S0268005X17306112-egi10HGQM8J72N.jpg' mode='r' encoding='UTF-8'>


In [ ]:
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

image_folder = gdrive_path
required_images = 190

# Create a dataset
dataset = datasets.ImageFolder(image_folder)

⚠️⚠️⚠️Uncomment this block if you want to experiment with ResNet 50. The model architecture takes a long time to render but gives a low accuracy ⚠️⚠️⚠️



In [ ]:
# import torch
# import torchvision.models as models
# import os
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader

# transform = transforms.Compose([
#     transforms.Resize((224, 224)),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

# dataset.transform = transform

# # Load pre-trained ResNet-50
# resnet = models.resnet50(pretrained=True)
# input_size = 224  # ResNet-50 input size

# # Remove the last layers (including the average pooling layer)
# modules = list(resnet.children())[:-2]

# # Create a new model
# feature_extractor = torch.nn.Sequential(*modules)

# # Add new layers
# feature_extractor.add_module('avg_pool', torch.nn.AdaptiveAvgPool2d((1, 1)))
# feature_extractor.add_module('flatten', torch.nn.Flatten())
# feature_extractor.add_module('fc', torch.nn.Linear(2048, len(dataset.classes)))
# feature_extractor.add_module('softmax', torch.nn.Softmax(dim=1))

# # Function to extract features
# def extract_features(dataset):
#     loader = DataLoader(dataset, batch_size=32, shuffle=False)
#     features = []
#     with torch.no_grad():
#         for i, (images, _) in enumerate(loader):
#             try:
#                 if not isinstance(images, torch.Tensor):
#                     print(f"Batch {i}: images is not a tensor. Type: {type(images)}")
#                     continue
#                 if images.dim() != 4:
#                     print(f"Batch {i}: images tensor has incorrect dimensions. Shape: {images.shape}")
#                     continue
#                 features.append(feature_extractor(images))
#             except Exception as e:
#                 print(f"Error in batch {i}: {str(e)}")
#     return torch.cat(features, dim=0) if features else None

# # Extract features
# features_train = extract_features(dataset)
# if features_train is None:
#     print("Failed to extract any features.")
# else:
#     print(f"Successfully extracted features. Shape: {features_train.shape}")

/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:01<00:00, 87.7MB/s]


KeyboardInterrupt: 

In [ ]:
# import torch
# import torchvision.models as models
# import os
# from torchvision import datasets, transforms
# from torch.utils.data import DataLoader
# output_folder = '/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/features_train_update2.pt'

# # Save the features
# torch.save(features_train, output_folder)


NameError: name 'features_train' is not defined

### Load the feature extraction from google drive

In [ ]:
import torch
import torchvision.models as models
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

output_folder = '/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/features_train.pt'

# Load the saved features
import torch
loaded_features = torch.load(output_folder)
print(f"Features loaded. Shape: {loaded_features.shape}")




Features loaded. Shape: torch.Size([5623, 29])


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

# Prepare labels
labels = [label for _, label in dataset]

# Divide data into training and validation sets
X_train, X_test, y_train, y_test = train_test_split(loaded_features, labels, test_size=0.2, random_state=42)

# Train an ensemble of classifiers (Random Forest in this case)
classifier = RandomForestClassifier(n_estimators=50, random_state=42)
classifier.fit(X_train, y_train)

# Test the classifier
predicted_labels = classifier.predict(X_test)
accuracy = accuracy_score(y_test, predicted_labels)
print(f'Accuracy: {accuracy * 100:.2f}%')

/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 25.69%


Pipeline for training again the ResNet 18 architecture and testing with Random Forest Classifer

- 1) Define a simple ResNet 18
- 2) Use Random Forest Classifier to predict
- 3) Modify ResNet 8 with more complicated architecture and transfer learning
- 4) Increase the number of trees
- 5) Pruning and Quantization to fit into a mobile app

In [ ]:
import torch
import torch.nn as nn
import torchvision.models as models
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

def trainResNet8(dataset, name):
    # Define transformations
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])
    dataset.transform = transform

    # Define Custom ResNet-8 Model
    class CustomResNet(nn.Module):
        def __init__(self, num_classes):
            super(CustomResNet, self).__init__()
            resnet = models.resnet18(pretrained=True)
            modules = list(resnet.children())[:-2]
            self.feature_extractor = nn.Sequential(*modules)
            self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
            self.flatten = nn.Flatten()
            self.fc = nn.Linear(512, num_classes)

        def forward(self, x):
            x = self.feature_extractor(x)
            x = self.avg_pool(x)
            x = self.flatten(x)
            x = self.fc(x)
            return x

    num_classes = len(dataset.classes)
    model = CustomResNet(num_classes=num_classes)

    # Fine-tune the last layer
    for param in model.parameters():
        param.requires_grad = False
    for param in model.fc.parameters():
        param.requires_grad = True

    # Define loss function and optimizer
    criterion = nn.CrossEntropyLoss()
    optimizer = torch.optim.Adam(model.fc.parameters(), lr=1e-4)

    # Train the model
    model.train()
    num_epochs = 5
    dataloader = DataLoader(dataset, batch_size=32, shuffle=True)
    for epoch in range(num_epochs):
        for images, labels in dataloader:
            outputs = model(images)
            loss = criterion(outputs, labels)
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item()}')

    # Extract features
    features_train = extract_features(model, dataset)
    if features_train is None:
        print("Failed to extract any features.")
    else:
        print(f"Successfully extracted features. Shape: {features_train.shape}")

    saveResNet8(torch, name, features_train)

    return features_train

def extract_features(model, dataset):
    model.eval()
    loader = DataLoader(dataset, batch_size=32, shuffle=False)
    features = []
    with torch.no_grad():
        for i, (images, _) in enumerate(loader):
            try:
                if not isinstance(images, torch.Tensor):
                    print(f"Batch {i}: images is not a tensor. Type: {type(images)}")
                    continue
                if images.dim() != 4:
                    print(f"Batch {i}: images tensor has incorrect dimensions. Shape: {images.shape}")
                    continue
                features.append(model(images))
            except Exception as e:
                print(f"Error in batch {i}: {str(e)}")
    return torch.cat(features, dim=0) if features else None

def saveResNet8(torch, name, features_train):
    output_folder = f'/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/{name}.pt'
    torch.save(features_train, output_folder)

def randomForestClassifierPrediction(dataset, loaded_features):
    # Prepare labels
    labels = [label for _, label in dataset]

    # Divide data into training and validation sets
    X_train, X_test, y_train, y_test = train_test_split(loaded_features, labels, test_size=0.2, random_state=42)

    # Train an ensemble of classifiers (Random Forest in this case)
    classifier = RandomForestClassifier(n_estimators=50, random_state=42)
    classifier.fit(X_train, y_train)

    # Test the classifier
    predicted_labels = classifier.predict(X_test)
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f'Accuracy: {accuracy * 100:.2f}%')
    return accuracy

def svm_classifier(dataset, loaded_features):
    # Prepare labels
    labels = [label for _, label in dataset]

    # Divide data into training and validation sets
    X_train, X_test, y_train, y_test = train_test_split(loaded_features, labels, test_size=0.2, random_state=42)

    # Train the SVM classifier
    classifier = SVC(kernel='linear', random_state=42)  # You can change the kernel to 'rbf', 'poly', etc., as needed
    classifier.fit(X_train, y_train)

    # Test the classifier
    predicted_labels = classifier.predict(X_test)

    # Find accuracy
    accuracy = accuracy_score(y_test, predicted_labels)
    print(f'Accuracy: {accuracy * 100:.2f}%')

    return classifier, accuracy


In [ ]:
from google.colab import drive
import os
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

drive.mount('/content/gdrive')
gdrive_path = '/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/Thang_folder/Herb_Images_Dataset_shortlisted'

image_folder = gdrive_path
required_images = 190

# Create a dataset
dataset = datasets.ImageFolder(image_folder)

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [12]:
# Usage usage
name = 'features_train_update2'
# comment the next line if you don't have the loaded features extracted
# features_train = trainResNet8(dataset, name)
loaded_features = torch.load(f'/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/{name}.pt')
randomForestClassifierPrediction(dataset, loaded_features)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 95.82%


0.9582222222222222

In [ ]:
# Example usage
name = 'features_train_update2'
# features_train = trainResNet8(dataset, name)
loaded_features = torch.load(f'/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/{name}.pt')
svm_classifier(dataset, loaded_features)


Accuracy: 97.78%


0.9777777777777777

In [ ]:
# Example usage
name = 'features_train_update2'
# features_train = trainResNet8(dataset, name)
loaded_features = torch.load(f'/content/gdrive/My Drive/Summer 2024/john hopskin/Training Herbal Data/model/{name}.pt')
classifier, accuracy = svm_classifier(dataset, loaded_features)


/usr/local/lib/python3.10/dist-packages/PIL/Image.py:996: UserWarning: Palette images with Transparency expressed in bytes should be converted to RGBA images
  warnings.warn(


Accuracy: 97.78%


Here are the steps to do the next stage: convert and save tensorflow lite for IOS app integration:


### Future Use
- Save the classifier: This step is needed because we need to save the classifier for future use.


### Conversion to Mobile Stage
The main function of these step is to let the IOS app can execute the code. The tensorflow approach can help to integrate both in IOS and Android. However, if there is any problem with this approach, we can just shift from implementing Tensorflow Lite to  CoreML's approach which only works for IOS.

#### Approach 1
- Convert the SVM classifier to Tensorflow Model

- Convert the Tensorflow Model to Tensorflow Lite model

- Save the tensorflow lite

#### Approach 2
- Convert to Omnx

- Convert from omnx to core ml

### Import Stage
- Import into IOS App



In [ ]:
# Save the classifier

import joblib

joblib.dump(classifier, 'svm_model.pkl')

['svm_model.pkl']

In [ ]:
from google.colab import files

files.download('svm_model.pkl')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
import tensorflow as tf
import numpy as np


# Define the function to convert the SVM model to a TensorFlow model
def svm_to_tf_model(classifier, input_shape):
    # Create a Sequential model
    model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(units=1, activation='linear', input_shape=input_shape)
    ])

    # Average the weights across support vectors
    weights = np.mean(classifier.coef_, axis=0)
    bias = classifier.intercept_[0]

    # Ensure weights shape matches the expected input shape
    print(f"Input shape: {input_shape}")
    print(f"Weight shape after averaging: {weights.shape}")

    expected_weight_shape = (input_shape[0], 1)
    if weights.size != expected_weight_shape[0]:
        raise ValueError(f"Expected weight shape {expected_weight_shape} but got {weights.shape}")

    model.layers[0].set_weights([weights.reshape(expected_weight_shape), np.array([bias])])

    return model

# Load your SVM model
classifier = joblib.load('svm_model.pkl')

# Define the input shape based on loaded features
input_shape = (loaded_features.shape[1],)

# Check the shapes
print(f"Loaded features shape: {loaded_features.shape}")
print(f"SVM classifier weights shape: {classifier.coef_.shape}")

# Convert to TensorFlow model
tf_model = svm_to_tf_model(classifier, input_shape)

# Save TensorFlow model
tf_model.save('svm_model.h5')

# Convert the TensorFlow model to TensorFlow Lite format
converter = tf.lite.TFLiteConverter.from_keras_model(tf_model)
tflite_model = converter.convert()

# Save the TensorFlow Lite model
with open('svm_model.tflite', 'wb') as f:
    f.write(tflite_model)




/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Loaded features shape: torch.Size([5623, 29])
SVM classifier weights shape: (406, 29)
Input shape: (29,)
Weight shape after averaging: (29,)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# Download the TensorFlow model
files.download('svm_model.h5')

# Download the TensorFlow Lite model
files.download('svm_model.tflite')

In [ ]:
# Pruning

### Second Stage: Build the mobile application. This will be done not in Jupyter Notebook.

https://chatgpt.com/c/eec94b74-1562-4597-ba23-62b96e1091a2


Link: https://www.youtube.com/watch?v=yV9nrRIC_R0

Identify the platform to develop the mobile application
Convert the model to Tensorflow Lite
Then integrate the TensorFlow Lite into the mobile application.



General Point about integrating the model into mobile application
 - Convert TensorFlow model to TensorFlow Lite
  - Apply TensorFlow Quantization to make the model efficient and compact to deploy on mobile.
  - Also, utilize the on-device accelerators to make the running of AI model faster.

- Think about how to integrate the model into the Flutter
  - Apply quantization to the ResNet 8 and autoencoder
  - Apply some conversions to the Random Forest Classifier
  - Apply image preproccessing before feature extract and dimensionality reduction. Finally, the use of RFC in classifying. .

- Build the mobile application with Flutter with simple UI/UX design with these following properties:
    - Allow user to upload the data
    - Then, the model will evaluate the data using the integrated ML model.
    - The model will give back the classification results and display on the app.
